In [1]:
import pandas as pd
import re
pd.__version__

'0.20.2'

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0, index_col=0)
components_df = df[0]
components_df.head(5)

,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
Symbol,,,,,,,
MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [3]:
# Drop columns SEC filings and CIK
components_df.drop(['SEC filings', 'CIK'], axis=1, inplace=True)
components_df.head(5)

# Convert column Date first added to a datetime column
components_df['Date first added'] = pd.to_datetime(components_df['Date first added'])

In [4]:
# Split Address of Headquarters into city and state
components_df[['City', 'State']] = components_df['Address of Headquarters'].str.split(',', expand=True, n=1)
components_df.State = components_df.State.str.strip()
components_df.head(5)

,Security,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,City,State
Symbol,,,,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaT,St. Paul,Minnesota
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,North Chicago,Illinois
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,North Chicago,Illinois
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,Dublin,Ireland
ATVI,Activision Blizzard,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,Santa Monica,California


In [5]:
# Drop the Address of Headquarters column
components_df.drop('Address of Headquarters', axis=1, inplace=True)
components_df.head(5)

,Security,GICS Sector,GICS Sub Industry,Date first added,City,State
Symbol,,,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,NaT,St. Paul,Minnesota
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31,North Chicago,Illinois
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31,North Chicago,Illinois
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,2011-07-06,Dublin,Ireland
ATVI,Activision Blizzard,Information Technology,Home Entertainment Software,2015-08-31,Santa Monica,California


In [6]:
components_df['GICS Sector'].value_counts()

Consumer Discretionary        85
Information Technology        69
Financials                    67
Industrials                   65
Health Care                   61
Consumer Staples              36
Energy                        34
Real Estate                   31
Utilities                     28
Materials                     25
Telecommunication Services     4
Name: GICS Sector, dtype: int64

In [7]:
# Most recently added to the index
components_df.loc[components_df['Date first added'].argmax()]

Security                 Align Technology
GICS Sector                   Health Care
GICS Sub Industry    Health Care Supplies
Date first added      2017-06-19 00:00:00
City                             San Jose
State                          California
Name: ALGN, dtype: object

In [8]:
# Added after a specific year, say, 2014
components_df.loc[components_df['Date first added'].dt.year > 2014].head(10)

,Security,GICS Sector,GICS Sub Industry,Date first added,City,State
Symbol,,,,,,
ATVI,Activision Blizzard,Information Technology,Home Entertainment Software,2015-08-31,Santa Monica,California
AYI,Acuity Brands Inc,Industrials,Electrical Components & Equipment,2016-05-03,Atlanta,Georgia
AMD,Advanced Micro Devices Inc,Information Technology,Semiconductors,2017-03-20,Sunnyvale,California
AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,2015-07-09,Roanoke,Virginia
ALK,Alaska Air Group Inc,Industrials,Airlines,2016-05-13,Seattle,Washington
ALB,Albemarle Corp,Materials,Specialty Chemicals,2016-07-01,Baton Rouge,Louisiana
ARE,Alexandria Real Estate Equities Inc,Real Estate,Office REITs,2017-03-20,Pasadena,California
ALGN,Align Technology,Health Care,Health Care Supplies,2017-06-19,San Jose,California
LNT,Alliant Energy Corp,Utilities,Electric Utilities,2016-07-01,Madison,Wisconsin


In [9]:
# Which state has the least concentration of S&P 500 companies and what are those companies?
least_represented_state = components_df.State.value_counts().argmin()
components_df.loc[components_df.State == least_represented_state]

,Security,GICS Sector,GICS Sub Industry,Date first added,City,State
Symbol,,,,,,
WYNN,Wynn Resorts Ltd,Consumer Discretionary,Casinos & Gaming,2008-11-14,Las Vegas,Nevada


In [10]:
# Which city serves as the headquarters of the most S&P 500 companies?
city_with_most_hqs = components_df.City.value_counts().argmax()
components_df.loc[components_df.City == city_with_most_hqs]

,Security,GICS Sector,GICS Sub Industry,Date first added,City,State
Symbol,,,,,,
AXP,American Express Co,Financials,Consumer Finance,1976-06-30,New York,New York
AIG,"American International Group, Inc.",Financials,Property & Casualty Insurance,1980-03-31,New York,New York
ARNC,Arconic Inc,Industrials,Aerospace & Defense,1964-03-31,New York,New York
AIZ,Assurant Inc,Financials,Multi-line Insurance,2007-04-10,New York,New York
BK,The Bank of New York Mellon Corp.,Financials,Asset Management & Custody Banks,NaT,New York,New York
BLK,BlackRock,Financials,Asset Management & Custody Banks,2011-04-04,New York,New York
BMY,Bristol-Myers Squibb,Health Care,Health Care Distributors,NaT,New York,New York
CBS,CBS Corp.,Consumer Discretionary,Broadcasting,1994-09-01,New York,New York
C,Citigroup Inc.,Financials,Diversified Banks,1988-05-31,New York,New York
